## Import packages

In [1]:
import os

import tensorflow as tf

from tfx import v1 as tfx
import tensorflow_transform.beam as tft_beam
from google.protobuf.json_format import MessageToDict
from tensorflow_transform.tf_metadata import dataset_metadata, schema_utils
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

import tempfile
import pprint
import warnings

pp = pprint.PrettyPrinter()

tf.get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore")

2023-07-30 01:24:10.292782: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-30 01:24:10.348690: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-30 01:24:10.349841: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 01:24:11.539261: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Define paths

In [2]:
_pipeline_root = './pipeline'

_data_root = './data/metro_traffic'

_data_filepath = os.path.join(_data_root, 'metro_traffic_volume.csv')

In [3]:
!head {_data_filepath}

holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,month,day,day_of_week,hour
None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,10,2,1,9
None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,10,2,1,10
None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,10,2,1,11
None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,10,2,1,12
None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,10,2,1,13
None,291.72,0.0,0.0,1,Clear,sky is clear,2012-10-02 14:00:00,5181,10,2,1,14
None,293.17,0.0,0.0,1,Clear,sky is clear,2012-10-02 15:00:00,5584,10,2,1,15
None,293.86,0.0,0.0,1,Clear,sky is clear,2012-10-02 16:00:00,6015,10,2,1,16
None,294.14,0.0,0.0,20,Clouds,few clouds,2012-10-02 17:00:00,5791,10,2,1,17


## Create InteractiveContext

In [4]:
context = InteractiveContext(pipeline_root = _pipeline_root)

### ExampleGen

ExampleGen splits the dataset into train and eval set (2/3 train, 1/3 eval). This dataset can be found in the pipeline directory.

In [5]:
example_gen = tfx.components.CsvExampleGen(input_base = _data_root)

In [6]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 16
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [7]:
artifact = example_gen.outputs['examples'].get()[0]
print(f'split names: {artifact.split_names}')
print(f'artifact uri : {artifact.uri}')
artifact_uri = artifact.uri

split names: ["train", "eval"]
artifact uri : ./pipeline/CsvExampleGen/examples/16


In [8]:
train_uri = os.path.join(artifact_uri, 'Split-train')
tfrecord_filenames = [os.path.join(train_uri, name)
                     for name in os.listdir(train_uri)]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type = 'GZIP')

2023-07-30 01:24:35.982615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-30 01:24:35.983228: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
def get_records(dataset, num_records):
    '''
    Extracts records from a given dataset
        Parameters:
            data : dataset saved by ExampleGen
            num_records : number of records to preview
        Returns:
            records : num_records number of records from the dataset 
    '''
    
    records = []
    for tfrecord in dataset.take(num_records):
        
        serialized_example = tfrecord.numpy()
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        example_dict = (MessageToDict(example))
        
        records.append(example_dict)
        
    return records

In [13]:
sample_records = get_records(dataset,3)

pp.pprint(sample_records)

[{'features': {'feature': {'clouds_all': {'int64List': {'value': ['40']}},
                           'date_time': {'bytesList': {'value': ['MjAxMi0xMC0wMiAwOTowMDowMA==']}},
                           'day': {'int64List': {'value': ['2']}},
                           'day_of_week': {'int64List': {'value': ['1']}},
                           'holiday': {'bytesList': {'value': ['Tm9uZQ==']}},
                           'hour': {'int64List': {'value': ['9']}},
                           'month': {'int64List': {'value': ['10']}},
                           'rain_1h': {'floatList': {'value': [0.0]}},
                           'snow_1h': {'floatList': {'value': [0.0]}},
                           'temp': {'floatList': {'value': [288.28]}},
                           'traffic_volume': {'int64List': {'value': ['5545']}},
                           'weather_description': {'bytesList': {'value': ['c2NhdHRlcmVkIGNsb3Vkcw==']}},
                           'weather_main': {'bytesList': {'value': 

2023-07-30 01:30:45.259063: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


### StatisticsGen

StatisticsGen computes the statistics over the dataset for analysis and validation.

In [14]:
statistics_gen = tfx.components.StatisticsGen(
    examples = example_gen.outputs['examples']
)

In [15]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 17
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [18]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

SchemaGen generates a schema based on data statistics.

In [19]:
schema_gen = tfx.components.SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 18
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [20]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'clouds_all',INT,required,,-
'date_time',BYTES,required,,-
'day',INT,required,,-
'day_of_week',INT,required,,-
'holiday',STRING,required,,'holiday'
'hour',INT,required,,-
'month',INT,required,,-
'rain_1h',FLOAT,required,,-
'snow_1h',FLOAT,required,,-


,Values
Domain,
'holiday',"'Christmas Day', 'Columbus Day', 'Independence Day', 'Labor Day', 'Martin Luther King Jr Day', 'Memorial Day', 'New Years Day', 'None', 'State Fair', 'Thanksgiving Day', 'Veterans Day', 'Washingtons Birthday'"
'weather_description',"'SQUALLS', 'Sky is Clear', 'broken clouds', 'drizzle', 'few clouds', 'fog', 'freezing rain', 'haze', 'heavy intensity drizzle', 'heavy intensity rain', 'heavy snow', 'light intensity drizzle', 'light intensity shower rain', 'light rain', 'light rain and snow', 'light shower snow', 'light snow', 'mist', 'moderate rain', 'overcast clouds', 'proximity shower rain', 'proximity thunderstorm', 'proximity thunderstorm with drizzle', 'proximity thunderstorm with rain', 'scattered clouds', 'shower drizzle', 'sky is clear', 'sleet', 'smoke', 'snow', 'thunderstorm', 'thunderstorm with heavy rain', 'thunderstorm with light drizzle', 'thunderstorm with light rain', 'thunderstorm with rain', 'very heavy rain', 'shower snow', 'thunderstorm with drizzle'"
'weather_main',"'Clear', 'Clouds', 'Drizzle', 'Fog', 'Haze', 'Mist', 'Rain', 'Smoke', 'Snow', 'Squall', 'Thunderstorm'"


### ExampleValidator

ExampleValidator detects anomalies in the data based on the schema generated.

In [22]:
example_validator = tfx.components.ExampleValidator(
    schema = schema_gen.outputs['schema'],
    statistics = statistics_gen.outputs['statistics']
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 19
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [23]:
context.show(example_validator.outputs['anomalies'])

### Transform 

In [24]:
_traffic_constants_module_file = 'traffic_constants.py'

In [25]:
%%writefile {_traffic_constants_module_file}

DENSE_FLOAT_FEATURE_KEYS = ['temp', 'snow_1h']
BUCKET_FEATURE_KEYS = ['rain_1h']
FEATURE_BUCKET_COUNT = {'rain_1h': 3}
RANGE_FEATURE_KEYS = ['clouds_all']

VOCAB_SIZE = 1000
OOV_SIZE = 10

VOCAB_FEATURE_KEYS = [
    'holiday',
    'weather_main',
    'weather_description'
]

CATEGORICAL_FEATURE_KEYS = [
    'hour', 'day', 'day_of_week', 'month'
]

VOLUME_KEY = 'traffic_volume'

def transformed_name(key):
    return key + '_xf'

Writing traffic_constants.py


In [26]:
_traffic_transform_module_file = 'traffic_transform.py'

In [28]:
%%writefile {_traffic_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import traffic_constants

_DENSE_FLOAT_FEATURE_KEYS = traffic_constants.DENSE_FLOAT_FEATURE_KEYS
_RANGE_FEATURE_KEYS = traffic_constants.RANGE_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = traffic_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = traffic_constants.VOCAB_SIZE
_OOV_SIZE = traffic_constants.OOV_SIZE
_CATEGORICAL_FEATURE_KEYS = traffic_constants.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = traffic_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = traffic_constants.FEATURE_BUCKET_COUNT
_VOLUME_KEY = traffic_constants.VOLUME_KEY
_transformed_name = traffic_constants.transformed_name

def preprocessing_fn(inputs):
    '''
    tf.transform's callback function for preprocessing inputs
    
        Parameters:
            inputs : map from feature keys to raw not-yet-transformed features
        Returns:
            outputs : Map from string feature key to transformed feature operations.
    '''
    
    outputs = {}
    
    for key in _DENSE_FLOAT_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_z_score(inputs[key])
        
    for key in _RANGE_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(inputs[key])
    
    for key in _VOCAB_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key], top_k=_VOCAB_SIZE, num_oov_buckets=_OOV_SIZE)
        
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(inputs[key], _FEATURE_BUCKET_COUNT[key])
        
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = inputs[key] 
    
   
    traffic_volume = tf.cast(inputs[_VOLUME_KEY], tf.float32)
    
    outputs[_transformed_name(_VOLUME_KEY)] = tf.cast(
        tf.greater(tft.mean(inputs[_VOLUME_KEY]), traffic_volume),
        tf.int64
    )
    
    return outputs

Overwriting traffic_transform.py


In [39]:
import traffic_transform

# traffic_transform.preprocessing_fn
(get_records(dataset, 1)[0]['features']['feature'])

2023-07-30 03:08:36.582816: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


{'holiday': {'bytesList': {'value': ['Tm9uZQ==']}},
 'weather_main': {'bytesList': {'value': ['Q2xvdWRz']}},
 'day_of_week': {'int64List': {'value': ['1']}},
 'weather_description': {'bytesList': {'value': ['c2NhdHRlcmVkIGNsb3Vkcw==']}},
 'traffic_volume': {'int64List': {'value': ['5545']}},
 'clouds_all': {'int64List': {'value': ['40']}},
 'rain_1h': {'floatList': {'value': [0.0]}},
 'day': {'int64List': {'value': ['2']}},
 'temp': {'floatList': {'value': [288.28]}},
 'date_time': {'bytesList': {'value': ['MjAxMi0xMC0wMiAwOTowMDowMA==']}},
 'snow_1h': {'floatList': {'value': [0.0]}},
 'hour': {'int64List': {'value': ['9']}},
 'month': {'int64List': {'value': ['10']}}}

In [46]:
raw_data = [
    {
        "holiday": "None",
        "temp": 273.67,
        "rain_1h": 0.0,
        "snow_1h": 0.13,
        "clouds_all": 90,
        "weather_main": "Snow",
        "weather_description": "light snow",
        "date_time": "2016-01-08 15:00:00",
        "traffic_volume": 5548,
        "month": 1,
        "day": 8,
        "day_of_week": 4,
        "hour": 15,
    }
]

feature_description = {
    "holiday": tf.io.FixedLenFeature([], tf.string),
    "temp": tf.io.FixedLenFeature([], tf.float32),
    "rain_1h": tf.io.FixedLenFeature([], tf.float32),
    "snow_1h": tf.io.FixedLenFeature([], tf.float32),
    "clouds_all": tf.io.FixedLenFeature([], tf.int64),
    "weather_main": tf.io.FixedLenFeature([], tf.string),
    "weather_description": tf.io.FixedLenFeature([], tf.string),
    "date_time": tf.io.FixedLenFeature([], tf.string),
    "traffic_volume": tf.io.FixedLenFeature([], tf.int64),
    "month": tf.io.FixedLenFeature([], tf.int64),
    "day": tf.io.FixedLenFeature([], tf.int64),
    "day_of_week": tf.io.FixedLenFeature([], tf.int64),
    "hour": tf.io.FixedLenFeature([], tf.int64),
}


#To study
import importlib
importlib.reload(traffic_transform)

raw_data_metadata = dataset_metadata.DatasetMetadata(schema_utils.schema_from_feature_spec(feature_description))

with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
    transformed_dataset, _ = (
        (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(traffic_transform.preprocessing_fn))

transformed_data, transformed_metadata = transformed_dataset

2023-07-30 03:12:37.985869: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_4' with dtype string and shape [?]
	 [[{{node inputs_4}}]]
2023-07-30 03:12:37.986019: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_11' with dtype string and shape [?]
	 [[{{node inputs_11}}]]
2023-07-30 03:12:37.986087: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_12' with dtype string and shape [?]
	 [[{{node inputs_12}}]]
2023-07-3

In [47]:
transformed_data

[{'clouds_all_xf': 1.0,
  'day_of_week_xf': 4,
  'day_xf': 8,
  'holiday_xf': 0,
  'hour_xf': 15,
  'month_xf': 1,
  'rain_1h_xf': 2,
  'snow_1h_xf': 0.0,
  'temp_xf': 0.0,
  'traffic_volume_xf': 0,
  'weather_description_xf': 0,
  'weather_main_xf': 0}]

In [48]:
MessageToDict(transformed_metadata.schema)

{'feature': [{'name': 'clouds_all_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'day_of_week_xf',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'day_xf',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'holiday_xf',
   'type': 'INT',
   'intDomain': {'isCategorical': True},
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'hour_xf',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'month_xf',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'rain_1h_xf',
   'type': 'INT',
   'intDomain': {'isCategorical': True},
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'snow_1h_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'temp_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'traffic_volume_xf',
 

In [49]:
transform = tfx.components.Transform(
    examples = example_gen.outputs['examples'],
    schema = schema_gen.outputs['schema'],
    module_file =  os.path.abspath(_traffic_transform_module_file)
)

context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying census_transform.py -> build/lib
copying census_constants.py -> build/lib
copying traffic_constants.py -> build/lib
copying util.py -> build/lib
copying traffic_transform.py -> build/lib
installing to /tmp/tmp7welzvfp
running install
running install_lib
copying build/lib/census_transform.py -> /tmp/tmp7welzvfp
copying build/lib/census_constants.py -> /tmp/tmp7welzvfp
copying build/lib/traffic_constants.py -> /tmp/tmp7welzvfp
copying build/lib/util.py -> /tmp/tmp7welzvfp
copying build/lib/traffic_transform.py -> /tmp/tmp7welzvfp
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manife

/home/abhipreets2/Tensorflow_Extended/py388/lib/python3.8/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+8abf0fc4e143d4ed4657bcd2ce2a014a2dcbc4072cb4d4838288c24d6203f95c-py3-none-any.whl



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+8abf0fc4e143d4ed4657bcd2ce2a014a2dcbc4072cb4d4838288c24d6203f95c-py3-none-any.whl



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+8abf0fc4e143d4ed4657bcd2ce2a014a2dcbc4072cb4d4838288c24d6203f95c-py3-none-any.whl



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
2023-07-30 03:17:06.419719: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_4' with dtype string and shape [?,1]
	 [[{{node inputs_4}}]]
2023-07-30 03:17:06.419929: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_11' with dtype string and shape [?,1]
	 [[{{node inputs_11}}]]
2023-07-30 03:17:06.420024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mu

ExecutionResult(
    component_id: Transform
    execution_id: 20
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [50]:
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(transform_graph_uri)

['transformed_metadata', 'transform_fn', 'metadata']

In [51]:
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

In [52]:
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [53]:
sample_records_xf = get_records(transformed_dataset, 3)
pp.pprint(sample_records_xf)

[{'features': {'feature': {'clouds_all_xf': {'floatList': {'value': [0.4]}},
                           'day_of_week_xf': {'int64List': {'value': ['1']}},
                           'day_xf': {'int64List': {'value': ['2']}},
                           'holiday_xf': {'int64List': {'value': ['0']}},
                           'hour_xf': {'int64List': {'value': ['9']}},
                           'month_xf': {'int64List': {'value': ['10']}},
                           'rain_1h_xf': {'int64List': {'value': ['2']}},
                           'snow_1h_xf': {'floatList': {'value': [-0.02742442]}},
                           'temp_xf': {'floatList': {'value': [0.5336852]}},
                           'traffic_volume_xf': {'int64List': {'value': ['0']}},
                           'weather_description_xf': {'int64List': {'value': ['4']}},
                           'weather_main_xf': {'int64List': {'value': ['0']}}}}},
 {'features': {'feature': {'clouds_all_xf': {'floatList': {'value': [0.75]}

2023-07-30 03:20:45.054114: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
